In [25]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score
from google.colab import drive

drive.mount('/content/drive')
path='/content/drive/MyDrive/data/train.csv'
train = pd.read_csv(path)
train.shape

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


(312795, 15)

In [26]:
test= '/content/drive/MyDrive/data/test.csv'
test=pd.read_csv(test)

test.shape
test.head(10)

,comment_text,severe_toxicity,obscene,sexual_explicit,identity_attack,insult,threat,other,gender,religion,race,disability,Target,processed_comment_text,comment_text_processed
0,So between the 2 civil lawyers going for the j...,0,0,0,0,0,0,1,0,0,0,0,0,"['civil', 'lawyer', 'going', 'job', 'one', 'st...",civil lawyer going job one stellar reputation ...
1,Hope they have bullet proof glass and bomb bar...,0,0,0,0,0,0,1,0,0,0,0,0,"['hope', 'bullet', 'proof', 'glass', 'bomb', '...",hope bullet proof glass bomb barrier well armed
2,"""...They realize the inter-connectedness betwe...",0,0,0,0,0,0,1,0,0,0,0,0,"['they', 'realize', 'interconnectedness', 'nat...",they realize interconnectedness nation world n...
3,"I'm a Raider fan, but I agree with Finley. Th...",0,0,0,0,0,0,1,0,0,0,0,0,"['raider', 'fan', 'agree', 'finley', 'these', ...",raider fan agree finley these player sit anthe...
4,I voted for Trump and it was not for any reaso...,0,0,0,0,0,0,1,0,0,0,0,0,"['voted', 'trump', 'reason', 'article', 'faceb...",voted trump reason article facebook what mains...
5,"My view is based on biology, not religion. I ...",0,0,0,0,0,0,1,1,0,0,0,0,"['view', 'based', 'biology', 'religion', 'real...",view based biology religion realized unborn ch...
6,Why does someone have to die before we take no...,0,0,0,0,0,0,1,0,0,0,0,0,"['why', 'someone', 'die', 'take', 'notice', 's...",why someone die take notice seriously injured ...
7,I would correct Ms Wente: There is ONE theocra...,0,0,0,0,0,0,1,0,0,0,0,0,"['would', 'correct', 'wente', 'there', 'one', ...",would correct wente there one theocratic antiw...
8,I wondered how long I'd have to scan before I ...,0,0,0,0,0,0,1,0,0,1,0,0,"['wondered', 'long', 'scan', 'got', 'abuse', '...",wondered long scan got abuse mlk word wonder m...
9,So he's a woman now? Mental Illness should be ...,0,0,0,0,0,0,1,1,0,0,0,0,"['woman', 'mental', 'illness', 'kept', 'discre...",woman mental illness kept discreet this much i...


In [27]:
train.info

<bound method DataFrame.info of                                              comment_text  severe_toxicity  \
0       Thousand of children abused by gay priests.\n\...                0   
1       Jim Minnery, Mike Dunleavy and any other so ca...                0   
2       This woman is too much of an arrogant know-it-...                0   
3       One name for a transgender bathroom is, "Sex o...                0   
4       IRT SpeakUp, when North Korea fires its first ...                0   
...                                                   ...              ...   
312790  yes.. Stunned.. Simply stunned.. \n\nThe part ...                0   
312791  You keep defending this by accusing CNN etc. o...                0   
312792  Local courts will be understanding and mild.  ...                0   
312793  baryton1, are you really suggesting that all M...                0   
312794  I have to question why Goodale is concerned ab...                0   

        obscene  sexual_explici

In [28]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [30]:
!pip install transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
train.head()

,comment_text,severe_toxicity,obscene,sexual_explicit,identity_attack,insult,threat,other,gender,religion,race,disability,Target,processed_comment_text,comment_text_processed
0,Thousand of children abused by gay priests.\n\...,0,0,0,1,0,0,0,1,0,0,0,1,"['thousand', 'child', 'abused', 'gay', 'priest...",thousand child abused gay priest this cost chu...
1,"Jim Minnery, Mike Dunleavy and any other so ca...",0,0,0,0,0,0,1,0,0,0,0,0,"['jim', 'minnery', 'mike', 'dunleavy', 'called...",jim minnery mike dunleavy called christian con...
2,This woman is too much of an arrogant know-it-...,0,0,0,0,1,0,0,1,0,0,0,1,"['this', 'woman', 'much', 'arrogant', 'knowita...",this woman much arrogant knowitall underpaid c...
3,"One name for a transgender bathroom is, ""Sex o...",0,0,0,0,0,0,1,1,0,0,0,0,"['one', 'name', 'transgender', 'bathroom', 'se...",one name transgender bathroom sex operation re...
4,"IRT SpeakUp, when North Korea fires its first ...",0,0,0,0,0,0,1,0,0,0,0,1,"['irt', 'speakup', 'north', 'korea', 'fire', '...",irt speakup north korea fire first rocket towa...


In [32]:
#Balancing the dataset by undersampling from the majority class
df_train_1 = train[train['Target']== 1]
df_train_0 = train[train['Target']== 0]
df_train_0_downsampled = df_train_0.sample(df_train_1.shape[0])
df_balanced = pd.concat([df_train_0_downsampled, df_train_1])
df_balanced['Target'].value_counts()

0    50715
1    50715
Name: Target, dtype: int64

In [34]:
sentences=df_balanced['comment_text']
labels=df_balanced['Target']
len(sentences),len(labels)

(101430, 101430)

In [47]:
#install the transformer models and load the bert base uncased model along with the tokenizer 
from transformers import BertTokenizer, TFBertForSequenceClassification

bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [48]:
#loop through the input sentences as part of the dataset and find the input ids and attention masks by storing it in an array
input_ids=[]
attention_masks=[]

for sent in sentences:
    bert_inp=bert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =64,pad_to_max_length = True,return_attention_mask = True)
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)
labels=np.array(labels)

len(input_ids),len(attention_masks),len(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


(101430, 101430, 101430)

In [36]:
# picke the input ids and labels along with the labels of the dataset
import pickle
print('Preparing the pickle file.....')

pickle_inp_path='/content/drive/MyDrive/data/bert_inp.pkl'
pickle_mask_path='/content/drive/MyDrive/data/bert_mask.pkl'
pickle_label_path='/content/drive/MyDrive/data/bert_label.pkl'

pickle.dump((input_ids),open(pickle_inp_path,'wb'))
pickle.dump((attention_masks),open(pickle_mask_path,'wb'))
pickle.dump((labels),open(pickle_label_path,'wb'))


print('Pickle files saved as ',pickle_inp_path,pickle_mask_path,pickle_label_path)

print('Loading the saved pickle files..')

input_ids=pickle.load(open(pickle_inp_path, 'rb'))
attention_masks=pickle.load(open(pickle_mask_path, 'rb'))
labels=pickle.load(open(pickle_label_path, 'rb'))

print('Input shape {} Attention mask shape {} Input label shape {}'.format(input_ids.shape,attention_masks.shape,labels.shape))

Preparing the pickle file.....
Pickle files saved as  /content/drive/MyDrive/data/bert_inp.pkl /content/drive/MyDrive/data/bert_mask.pkl /content/drive/MyDrive/data/bert_label.pkl
Loading the saved pickle files..
Input shape (101430, 64) Attention mask shape (101430, 64) Input label shape (101430,)


In [37]:
#split the dataset into train(80%) and validation(20%)
from sklearn.model_selection import train_test_split
train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.2)

print('Train inp shape {} Val input shape {}\nTrain label shape {} Val label shape {}\nTrain attention mask shape {} Val attention mask shape {}'.format(train_inp.shape,val_inp.shape,train_label.shape,val_label.shape,train_mask.shape,val_mask.shape))

Train inp shape (81144, 64) Val input shape (20286, 64)
Train label shape (81144,) Val label shape (20286,)
Train attention mask shape (81144, 64) Val attention mask shape (20286, 64)


In [49]:
#summarizes the bert model used in this project
bert_model.summary()

Model: "tf_bert_for_sequence_classification_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_113 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [50]:
#define the loss parameter, ADAM Optimizer and metrics to compile the model 
import tensorflow as tf
import keras
log_dir='tensorboard_data/tb_bert'
model_save_path='./models/bert_model.h5'

print('\nBert Model',bert_model.summary())


from tensorflow.keras.optimizers.legacy import Adam
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')]
optimizer = Adam(learning_rate=2e-5,epsilon=1e-8)

bert_model.compile(loss=loss,optimizer=optimizer,metrics=metrics)

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]

Model: "tf_bert_for_sequence_classification_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_113 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________

Bert Model None


In [51]:
#fit the BERT model using the training dataset for 3 epochs and batch size of 64
history=bert_model.fit([train_inp,train_mask],train_label,batch_size=64,epochs=3,validation_data=([val_inp,val_mask],val_label))

Epoch 1/3
1268/1268 [==============================] - 1214s 943ms/step - loss: 0.4259 - accuracy: 0.8029 - val_loss: 0.3952 - val_accuracy: 0.8220
Epoch 2/3
1268/1268 [==============================] - 1142s 901ms/step - loss: 0.3437 - accuracy: 0.8463 - val_loss: 0.3931 - val_accuracy: 0.8235
Epoch 3/3
1268/1268 [==============================] - 1194s 942ms/step - loss: 0.2693 - accuracy: 0.8832 - val_loss: 0.4757 - val_accuracy: 0.8134


In [52]:
test_sentences=test['comment_text']
test_labels=test['Target']
len(test_sentences),len(test_labels)




(134056, 134056)

In [53]:
#similar to geenrating the input ids and attention masks for training, we follow a similar approach for test dataset as well
test_input_ids=[]
test_attention_masks=[]
test_bert_inp=[]
for sent in test_sentences:
    bert_inp=bert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =64,pad_to_max_length = True,return_attention_mask = True)
    test_bert_inp.append(bert_inp)
    test_input_ids.append(bert_inp['input_ids'])
    test_attention_masks.append(bert_inp['attention_mask'])

test_input_ids=np.asarray(test_input_ids)
test_attention_masks=np.array(test_attention_masks)
test_labels=np.array(test_labels)

len(test_input_ids),len(test_attention_masks),len(test_labels)


(134056, 134056, 134056)

In [55]:
#predict the values by using the trained BERT model and test dataset
test_preds = bert_model.predict([test_input_ids, test_attention_masks])
test_preds1 = np.argmax(test_preds, axis=-1)




4190/4190 [==============================] - 626s 148ms/step


In [65]:
test_preds[0]

probs = tf.nn.sigmoid(test_preds[0])
print(probs)
test_preds1 = np.argmax(test_preds[0], axis=-1)
print(len(test_preds1))

tf.Tensor(
[[0.8274785  0.20153068]
 [0.334586   0.60781485]
 [0.73877364 0.26146433]
 ...
 [0.9651083  0.0413583 ]
 [0.6045871  0.39454654]
 [0.7022489  0.27350396]], shape=(134056, 2), dtype=float32)
134056


In [66]:
#evaluate the output generated by bert using accuracy, precision, recall and F1 score. 
print(test_preds1)
test['predicted_values'] = test_preds1
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score 
test_acc = accuracy_score(test_labels, test_preds1)
test_f1_score = f1_score(test_labels, test_preds1)
test_recall_score = recall_score(test_labels, test_preds1)
test_precision_score = precision_score(test_labels, test_preds1)
print("Accuracy: {:.2f}%".format(test_acc))
print("Precision: {:.2f}%".format(test_f1_score))
print("Recall: {:.2f}%".format(test_recall_score))
print("F1 score: {:.2f}%".format(test_precision_score))

[0 1 0 ... 0 0 0]
Accuracy: 0.77%
Precision: 0.55%
Recall: 0.86%
F1 score: 0.41%


In [67]:
test.head(20)

,comment_text,severe_toxicity,obscene,sexual_explicit,identity_attack,insult,threat,other,gender,religion,race,disability,Target,processed_comment_text,comment_text_processed,predicted_values
0,So between the 2 civil lawyers going for the j...,0,0,0,0,0,0,1,0,0,0,0,0,"['civil', 'lawyer', 'going', 'job', 'one', 'st...",civil lawyer going job one stellar reputation ...,0
1,Hope they have bullet proof glass and bomb bar...,0,0,0,0,0,0,1,0,0,0,0,0,"['hope', 'bullet', 'proof', 'glass', 'bomb', '...",hope bullet proof glass bomb barrier well armed,1
2,"""...They realize the inter-connectedness betwe...",0,0,0,0,0,0,1,0,0,0,0,0,"['they', 'realize', 'interconnectedness', 'nat...",they realize interconnectedness nation world n...,0
3,"I'm a Raider fan, but I agree with Finley. Th...",0,0,0,0,0,0,1,0,0,0,0,0,"['raider', 'fan', 'agree', 'finley', 'these', ...",raider fan agree finley these player sit anthe...,0
4,I voted for Trump and it was not for any reaso...,0,0,0,0,0,0,1,0,0,0,0,0,"['voted', 'trump', 'reason', 'article', 'faceb...",voted trump reason article facebook what mains...,0
5,"My view is based on biology, not religion. I ...",0,0,0,0,0,0,1,1,0,0,0,0,"['view', 'based', 'biology', 'religion', 'real...",view based biology religion realized unborn ch...,1
6,Why does someone have to die before we take no...,0,0,0,0,0,0,1,0,0,0,0,0,"['why', 'someone', 'die', 'take', 'notice', 's...",why someone die take notice seriously injured ...,0
7,I would correct Ms Wente: There is ONE theocra...,0,0,0,0,0,0,1,0,0,0,0,0,"['would', 'correct', 'wente', 'there', 'one', ...",would correct wente there one theocratic antiw...,0
8,I wondered how long I'd have to scan before I ...,0,0,0,0,0,0,1,0,0,1,0,0,"['wondered', 'long', 'scan', 'got', 'abuse', '...",wondered long scan got abuse mlk word wonder m...,0
9,So he's a woman now? Mental Illness should be ...,0,0,0,0,0,0,1,1,0,0,0,0,"['woman', 'mental', 'illness', 'kept', 'discre...",woman mental illness kept discreet this much i...,0


In [69]:
test.to_csv('/content/drive/MyDrive/data/train.csv', index=False)